# Data Wrangling

* Merging Datasets together
* Pivoting data
* Grouping 


In [ ]:
import pandas as pd
import numpy as np

## Merging Data

* Bringing disparate datasets together is one of the more powerful features of Pandas
* Like with Python lists, you can `append()` and `concat()` Pandas `Series` and `Dataframes`
* The `concat` is a module function, you call it directly from the pandas module (usually called `pd`)

In [ ]:
# concatinate two series together
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2]) #note the Seres are passed as a list

In [ ]:
# order matters
pd.concat([ser2, ser1])

In [ ]:
# concatinate two dataframes
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2])

* Pandas will automatically line up matching indexes

In [ ]:
# concatinate dataframes horizontally
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"C":["C1", "C2"],
                    "D":["D1","D2"]},index=[1,2])
pd.concat([df1,df2], axis=1)

* And pandas will gracefully handle mis-alignment

In [ ]:
# What happens when indexes don't line up
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2], axis=1)

* The `append` function is a method of a Series/Dataframe and returns a new object

In [ ]:
# append df2 to df1
df1.append(df2)

### Merging and Joining

* While `concat()` is useful it lacks the power to do complex data merging
* For example, I have two tables of different data but one shared column
* This is where the `merge()` function becomes useful because it lets you *join* datasets
* The concept of "join" has lots of theory and is a richly developed method for *joining* data

#### One-to-one joins

In [ ]:
# create two dataframes with one shared column
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', "Nancy"],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', "Librarian"]})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [ ]:
# display df1
df1

In [ ]:
# display df2
df2

In [ ]:
# merge df1 and df2 into a new dataframe df3
df3 = pd.merge(df1, df2)
df3

* The new dataframe `df3` now has all of the data from df1 and df2
* The `merge` function automatically connected the two tables on the "employee" column
* But what happens when your data don't line up?

#### Many-to-one joins

* Sometimes there isn't a one to one relationshp between rows in  two datasets
* A *many-to-one* join lets you combine these datasets

In [ ]:
df3

In [ ]:
# make another dataframe about the supervisor for each group
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

In [ ]:
# Merge df3 from above with the supervisor info in df4
pd.merge(df3,df4)

* Notice how the information about Guido, the manager for Engineering, is repeated.
* Pandas automatically fills in these values to maintain the tabular, 2 dimensional structure of the data
* While this might seem like duplicated data, it makes it easier to quickly look up Jake and Lisa's supervisor without consulting multiple tables

#### Many-to-many joins

* Let's combine the employee information with skills information
* Notice there isn't a one to one or even a one to many relationship between these tables
* Each group can have multiple skills, so **what do you think will happen?**

In [ ]:
# Use the employee table specified above
df1

In [ ]:
# create a new dataframe with skills information
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'Librarian'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'nunchucks']})
df5

In [ ]:
pd.merge(df1, df5)

* Amazing, Pandas merge capabilities are very useful when column names match up
* But what do you do if the names of your columns don't match?
* You could change column names...
* But that is crazy! Just use the `left_on` and `right_on` parameters to the `merge()` function

In [ ]:
# Use the employee table specified above
df1

In [ ]:
df2 = df2.rename({"employee":"name"}, axis="columns")

In [ ]:
# lets try and merge them without specifying what to merge on
pd.merge(df1, df2)

* Gak, error! Pandas can't figure out how to combine them
* What are the column names I should specify?

In [ ]:
# Now lets specify the column name 
pd.merge(df1, df2, left_on="employee", right_on="name" )

* Notice we now have a redundant employee/name column, this is a by-product of merging different columns
* If you want to get rid of it you can use the `drop` method

In [ ]:
# drop the name column, axis=1 means axis='col', which is confusing
pd.merge(df1, df2, left_on="employee", right_on="name" ).drop("name", axis=1)

* This is just a taste of merging and joining data
* We will cover more of this in the SQL and Relational Databases sessions

## Pivoting Data

* Sometimes you get what is called "long" or "stacked" data (streaming values from an instrument or periodic observational data)
* Data in this shape can be difficult to analyze

In [ ]:
# load the CSV file
data = pd.read_csv("../4 - data management one/community-center-attendance.csv",
                  index_col="_id")
                   
# look at the first ten rows of the data
data.head(10)

In [ ]:
# How many rows we got?
data.shape

* These data are looooooong
* Each row represents a community center in Pittsburgh reporting how many people visited the center
* Given this shape it is possible to do some calculations, but it might make more sense to *pivot* the data so that each column is a community center and each row is a day

In [ ]:
# Use the pivot function to make column values into columns
pivoted_data = data.pivot_table(index="date", # these values will be rows
                          columns="center_name", # these values will be columns
                          values="attendance_count" # these values will populate the table
                         )
pivoted_data.head()

* Now we can easily find out things about our favorite community centers

In [ ]:
# Total number of people who have visited Magee
pivoted_data['Magee Community Center'].sum()

In [ ]:
# Average attendence per day at Magee
pivoted_data['Magee Community Center'].mean()

### Transpose

* Pandas has a handy function for *transposing* dataframes
* It just rotates the table making the columns rows and the rows columns

In [ ]:
pivoted_data.T

* Now the Column and row indexes are swapped

## Grouping Data


* A common pattern in data analysis is splitting data by a key and then performing some math on all of the values with that key and finally combining it all back together
* This is commonly known in data circles as *split, apply, combine*


In [ ]:
# create a dataframe to illustrate GroupBy
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6),
                   'counts':[45,234,6,2,1324,345], 
                   'things':['dog', 'cat', 'cat', 'dog', 'cat', 'cat']}
                 )
df

In [ ]:
# Dataframes have a method, groupby(), that takes a column name be be the grouping key
df.groupby('key')

In [ ]:
for group in df.groupby('key'):
    print("Group for key:", group[0])
    print("Data:", group[1])
    print("Data Type:", type(group[1]))
    print()

* Cool, we can see that we have *split* our data into three groups
* Now, we need to tell Pandas what function to *apply* to each group
* We need to specify what kind of aggregation, transformation, or computation to perform on the group

In [ ]:
# Tell pandas to add up all of the values for each key
df.groupby('key').sum()

* Under the hood Pandas is creating a bunch of new Dataframes based on the grouping column values

In [ ]:
# you can save the group object and run different aggregations
grouped_dataframe = df.groupby('key')
grouped_dataframe.sum()

In [ ]:
grouped_dataframe.mean()

* The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``size()``               | Total number of items w/ NaNs   |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

* These are all methods of ``DataFrame`` and ``Series`` objects.

* You can also do multiple levels of grouping

In [ ]:
df.groupby(['things','key']).count()

* What you are seeing is what is called a [Multilevel Index](https://pandas.pydata.org/pandas-docs/stable/advanced.html)
* Sadly, we don't have time to cover that topic, but this chapter on [Hierarchical Indexing](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html) in the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) is a great introduction to the topic.

## Split, Apply, Combine with real data

* Lets grab a dataset from the WPRDC, the [Allegheny County Jail Daily Census](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census)

In [ ]:
# Grab three months of data
january17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/3b5d9c45-b5f4-4e05-9cf1-127642ad1d17")
feburary17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/cb8dc876-6285-43a8-9db3-90b84eedb46f")
march17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/68645668-3f89-4831-b1de-de1e77e52dd3")


In [ ]:
january17_jail_census.head()

* Use the concat function to combine all three into one dataframe
* Remember `concat` is a general pandas function so we call it with `pd.concat`
* It takes as an argument a list of things to combine

In [ ]:
# make a list of dataframes to combine
dataframes = [january17_jail_census, 
              feburary17_jail_census, 
              march17_jail_census]

# give the concat function the list
jail_census = pd.concat(dataframes)
jail_census

* Now we can do some calculations on the data
* Note, because this is a daily census it includes many of the same people, so this mean isn't statistically *meaningful* 
* We can still use these data for demonstration purposes 

In [ ]:
# Compute the average age ate booking by gender
jail_census.groupby('Gender')['Age at Booking'].mean()

In [ ]:
# compute the average age at booking by race then gender 
jail_census.groupby(['Race', 'Gender'])['Age at Booking'].mean()

If we look at the [data dictionary](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census/resource/f0550174-16b0-4f6e-88dc-fa917e74b56c) we can see the following mapping for race categories
```
Race of Inmate
A-ASIAN OR PACIFIC ISLANDER
B-BLACK OR AFRICAN AMERICAN
H-HISPANIC 
I-AMERICAN INDIAN OR ALASKAN NATIVE
U-UNKNOWN
W-WHITE
```


In [ ]:
# Get the statistical summary of age at booking by gender
jail_census.groupby('Gender')['Age at Booking'].describe()

In [ ]:
# Compute the difference between Age at Booking and current age
age_difference = jail_census['Current Age'] - jail_census['Age at Booking']
age_difference.value_counts()

* This uses a mathematical funciton to compute the age difference for each row
* Then we use `value_counts` to count the age differences

In [ ]:
jail_census.groupby('Date').count()

In [ ]:
jail_census['year'] = jail_census['Date'].str.split("-").str[0]
jail_census['month'] = jail_census['Date'].str.split("-").str[1]
jail_census['day'] = jail_census['Date'].str.split("-").str[2]

jail_census.head()

In [ ]:
jail_census.groupby('month').count()

In [ ]:
jail_census.groupby('day').count()

* This is a really awkward way of dealing with time
* We shouldn't have to make a separate column for year, month, day
* There must be a better way to do this time stuff...
